In [ ]:
!mkdir ~\.kaggle

In [ ]:
!cd ~\.kaggle && ls

In [ ]:
!kaggle competitions download -c petfinder-pawpularity-score

In [ ]:
import tensorflow as tf
from keras import backend as K 

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
def limit_mem():
    K.get_session().close()
    cfg = K.tf.compat.v1.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.compat.v1.Session(config=cfg))

In [2]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.config.run_functions_eagerly(False)


In [3]:
#import af libaries
import tensorflow as tf
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow_addons as tfa
from PIL import Image
import imagehash
import imageio
import random
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers



In [4]:
#test is not working i need to have a look at the download again
train = pd.read_csv('train.csv')
#test = pd.read_csv('test.csv')
print("Amount of tranning's data: " + str(len(train)))
#print("Amount of test data: " + str(len(test)))
image_size = 100


Amount of tranning's data: 9912


In [5]:
#Loading images
def get_train_file_path(image_id):
    return r"C:\Users\tobia\GitHub\PetFinder\train\{}.jpg".format(image_id)

def get_test_file_path(image_id):
    return r"C:\Users\tobia\GitHub\PetFinder\test\{}.jpg".format(image_id)

train['file_path'] = train['Id'].apply(get_train_file_path)
#test['file_path'] = test['Id'].apply(get_test_file_path)


# Shows the first 5, change the paramter for more
display(train.head())


,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,file_path
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,C:\Users\tobia\GitHub\PetFinder\train\0007de18...
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,C:\Users\tobia\GitHub\PetFinder\train\0009c66b...
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,C:\Users\tobia\GitHub\PetFinder\train\0013fd99...
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,C:\Users\tobia\GitHub\PetFinder\train\0018df34...
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,C:\Users\tobia\GitHub\PetFinder\train\001dc955...


In [ ]:
#Visualizing the spread??
train.hist(figsize=(15,15))
plt.tight_layout()
plt.show()

In [21]:
#Mean of Pawpularity remove [] if you want the see mean on everything
print("Mean value of train "+ str(train['Pawpularity'].mean()))
train.info()

Mean value of train 37.98927985874637
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7929 entries, 2035 to 7475
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id             7929 non-null   object
 1   Subject Focus  7929 non-null   int64 
 2   Eyes           7929 non-null   int64 
 3   Face           7929 non-null   int64 
 4   Near           7929 non-null   int64 
 5   Action         7929 non-null   int64 
 6   Accessory      7929 non-null   int64 
 7   Group          7929 non-null   int64 
 8   Collage        7929 non-null   int64 
 9   Human          7929 non-null   int64 
 10  Occlusion      7929 non-null   int64 
 11  Info           7929 non-null   int64 
 12  Blur           7929 non-null   int64 
 13  Pawpularity    7929 non-null   int64 
 14  file_path      7929 non-null   object
dtypes: int64(13), object(2)
memory usage: 991.1+ KB


In [ ]:
#Kinda slow might be a faster way to do it.
#It gets height and width of each image and then calculate the aspect ratio
widths = []
heights = []
ratios = []
for file_path in tqdm(train['file_path']):
    image = imageio.imread(file_path)
    h, w, _ = image.shape
    heights.append(h)
    widths.append(w)
    ratios.append(w / h)

In [ ]:
#First we plot it to see the values
plt.figure(figsize=(15,8))
plt.title(f'Height and Width', size=24)
plt.hist(heights, bins=32, label='Heights')
plt.hist(widths, bins=32, label='Widths')
plt.legend(prop={'size': 16})
plt.show()

In [ ]:
#https://www.kaggle.com/markwijkhuizen/petfinder-eda-yolov5-obj-detection-tfrecords#Image-EDA
display(pd.Series(ratios).describe())
plt.figure(figsize=(15,8))
plt.title(f'Ratios', size=24)
plt.hist(ratios, bins=15, label='Ratios')
plt.legend(prop={'size': 15})
plt.show()

In [ ]:
#Hard to see the coalition on the ???bar??? graph so we create a scatter point graph
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
points = ax.scatter(widths, heights, color='blue', alpha=0.5, s=ratios, picker=True)
ax.set_title("Image Resolution")
ax.set_xlabel("Width", size=14)
ax.set_ylabel("Height", size=14)

In [ ]:
#Shows 5x5 images and their title is their paw score
plt.figure(figsize=(20, 20))
for i in range(5 * 5):
    plt.subplot(5, 5, i+1)
    image = plt.imread(train.iloc[i, -1])
    #image = plt.cvtColor(image, cv2.COLOR_BGR2RGB)
    Pawpularity = train.iloc[i, -2]
    plt.imshow(image)
    plt.title(f"Pawpularity Score: {Pawpularity}")
plt.show()

In [ ]:
#ligner der er duplicates som skal fjernes 
#https://www.kaggle.com/c/petfinder-pawpularity-score/discussion/278497 kode til at fjerne dem findes her
tqdm.pandas()
def get_hash(file_path):
    img = Image.open(file_path)
    img_hash = imagehash.phash(img)
    
    return img_hash.hash.reshape(-1).astype(np.uint8)
    
train['phash'] = train['file_path'].progress_apply(get_hash)

In [ ]:
def find_similar_images(threshold=0.90):
    # Number of Duplicate Images Found
    duplicate_counter = 1
    # Indices of Duplicate Images
    duplicate_idxs = set()
    # For each image in the train dataset
    for idx, phash in enumerate(tqdm(train['phash'])):
        # Compute the similarity to all other images
        for idx_other, phash_other in enumerate(train['phash']):
            # Similarity score is imply the percentage of equal bits
            similarity = (phash ==phash_other).mean()
            # Prevent self comparison, threshold similarity and ignore repetetive duplicate detection
            if idx != idx_other and similarity > threshold and not(duplicate_idxs.intersection([idx, idx_other])):
                # Update Duplicate Indices
                duplicate_idxs.update([idx, idx_other])
                # Get DataFrame rows
                row = train.loc[idx]
                row_other = train.loc[idx_other]
                # Plot Duplicate Images
                fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8,5))
                ax[0].imshow(imageio.imread(row['file_path']))
                ax[0].set_title(f'Idx: {idx}, Pawpularity: {row["Pawpularity"]}')
                ax[1].imshow(imageio.imread(row_other['file_path']))
                ax[1].set_title(f'Idx: {idx_other}, Pawpularity: {row_other["Pawpularity"]}')
                plt.suptitle(f'{duplicate_counter} | PHASH Similarity: {similarity:.3f}')
                plt.show()
                # Increase Duplicate Counter
                duplicate_counter += 1
                
    # Return Indices of Duplicates
    return duplicate_idxs
    
duplicate_idxs = find_similar_images()
#The same images don't have the same paw score. The majority's paw score's diffrence is around 10.
#But some got over double the score'

In [ ]:
print(f'Found {len(duplicate_idxs)} Duplicate Images')
# Removing Duplicate Images, 
#Code taken from https://www.kaggle.com/markwijkhuizen/petfinder-eda-yolov5-obj-detection-tfrecords#Image-EDA
#IMPORTANT(I think so atleast)! If this code is run twice it will drop both images instead of just one of the duplicates
#Hhich image is correct one? They got diffrent scores
train = train.drop(duplicate_idxs).reset_index(drop=True)
print("Amount of tranning's data: " + str(len(train)))

In [ ]:
# Shows a batch of images we will use this function to see the lowest and highest pawpularity
def show_batch_df(df, rows=6, cols=4):
    df = df.copy().reset_index()
    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(cols*4, rows*4))
    for r in range(rows):
        for c in range(cols):
            idx = r * cols + c
            img = imageio.imread(df.loc[idx, 'file_path'])
            plt.imshow(img)
            axes[r, c].set_title(f'{idx}, label: {df.loc[idx, "Pawpularity"]}')
            


In [ ]:
#Images with the lowest value
sorted_lowest = (train.sort_values('Pawpularity'))
plt.figure(figsize=(20, 20))
for i in range(5 * 5):
    plt.subplot(5, 5, i+1)
    image = plt.imread(sorted_lowest.iloc[i, -1])
    Pawpularity = sorted_lowest.iloc[i, -2]
    plt.imshow(image)
    plt.title(f"Pawpularity Score: {Pawpularity}")
plt.show()

In [ ]:
#Images with the highest value
sorted_highest = (train.sort_values('Pawpularity',ascending=False))
plt.figure(figsize=(20, 20))
for i in range(5 * 5):
    plt.subplot(5, 5, i+1)
    image = plt.imread(sorted_highest.iloc[i, -1])
    Pawpularity = sorted_highest.iloc[i, -2]
    plt.imshow(image)
    plt.title(f"Pawpularity Score: {Pawpularity}")
plt.show()

In [6]:
#We need some more preproccsing and data normalization
def dataNormalization(image):
    image = Image.open(image)
    #tf.image.convert_image_dtype(image, tf.float32)
    #image = tf.reshape(image, [352, 352, 3])
    image = tf.image.resize(image, (image_size, image_size))
    image = tf.cast(image, tf.float32) / 255.0
    return image

In [7]:
#train['file_path'] = train['file_path'].apply(dataNormalization)
#image = plt.imread(train.loc[0,'file_path'])
#ds = tf.data.Dataset.from_tensor_slices(dict(train))
AUTOTUNE = tf.data.experimental.AUTOTUNE

#opdel i test og train
train, test = train_test_split(train, test_size=0.2)
print("Amount of tranning's data: " + str(len(train)))
print("Amount of test data: " + str(len(test)))

train_dataset = train['file_path'].map(dataNormalization)
test_dataset = test['file_path'].map(dataNormalization)


Amount of tranning's data: 7929
Amount of test data: 1983


In [8]:
#checks if the values are normalized
#print(train_dataset[0])
#we need proper names here
train_pred = train.loc[:,'Pawpularity']
test_pred = test.loc[:,'Pawpularity']


In [ ]:
print(train_pred)
#type(pred)
#result = pd.merge(train_dataset, pred, left_on=None)
#train_dataset = pd.merge(train_dataset, train_pred, left_index=True, right_index=True)
#test_dataset = pd.merge(test_dataset, test_pred, left_index=True, right_index=True)

In [ ]:
print(train_dataset)

In [ ]:
display(train_dataset.head())

#print(train_dataset['file_path'])
#display(result.head())

In [ ]:
plt.figure(figsize=(20, 20))
for i in range(5 * 5):
    plt.subplot(5, 5, i+1)
    plt.title('Original image')
    image =  train_dataset.iloc[i]
    plt.imshow(image)
plt.show(block=False)

In [ ]:
#Side by side comparrison with the new ratio
for i in range(5 * 5):
    result_image = train_dataset.iloc[i]
    train_image = plt.imread(train.iloc[i,-1])
    visualize(train_image,result_image)

In [9]:
def augment(augmented):
    # We need to test with the diffrent type of augment to see if they hurt the acc. 
    # We should also take some of the fuctions we used in CNN
    augmented = tf.image.random_brightness(augmented, 0.05)
    augmented = tf.image.random_flip_left_right(augmented)
    augmented = tf.image.random_saturation(augmented, 0.95, 1)
    augmented = tf.image.random_contrast(augmented, 0.95, 1)
    augmented = tf.image.random_hue(augmented, 0.05)
    augmented = shear(augmented)
    return augmented

def visualize(original, augmented):
    fig = plt.figure()
    plt.subplot(1,2,1)
    plt.title('Original image')
    plt.imshow(original)

    plt.subplot(1,2,2)
    plt.title('Augmented image')
    plt.imshow(augmented)
    
def shear(image):
    #shear
    #de her værdier skal nok ændres i
    random_number_shear = random.uniform(-0.2,0.2)
    shear = random_number_shear
    shear_rotate = shear * -20
    image = tfa.image.transform(image, [1.0, shear, shear_rotate, 0.0, 1.0, 0.0, 0.0, 0.0])
    return image

def crop(image):
    # Add 6 pixels of padding
    image = tf.image.resize_with_crop_or_pad(image, img_height + 6, img_width + 6) 
    # Random crop back to the original size
    image = tf.image.random_crop(image, size=[img_height, img_width, 1])
    return image

In [ ]:
for i in range(3):
    image = train_dataset.iloc[i]
    changed_image = augment(image)
    visualize(image,changed_image)

In [ ]:
#worse model
model = tf.keras.models.Sequential([
layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
layers.MaxPool2D(pool_size=2, strides=2),
layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
layers.MaxPool2D(pool_size=2, strides=2),
layers.Flatten(),
layers.Dense(units=128, activation='relu'),
layers.Dense(units=1)
])

In [10]:
#First model
model = tf.keras.models.Sequential([
layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
layers.Conv2D(filters=64, kernel_size=3, activation='relu'),
layers.MaxPool2D(pool_size=2, strides=2),
layers.Dropout(0.25),
layers.Conv2D(filters=128, kernel_size=3, activation='relu'),
layers.MaxPool2D(pool_size=2, strides=2),
layers.Conv2D(filters=128, kernel_size=3, activation='relu'),
layers.MaxPool2D(pool_size=2, strides=2),
layers.Dropout(0.25),
layers.Flatten(),
layers.Dense(units=1024, activation='relu'),
layers.Dropout(0.50),
layers.Dense(units=1)
])



In [11]:
model.compile(loss="mse", 
              optimizer = tf.keras.optimizers.Adam(), 
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [12]:
#print(train_dataset.shape)
#print(test_dataset.shape)
#print(train_dataset)
def labelNormalize(value):
    return value/100;
#train_pred = train_pred.map(labelNormalize)
#test_pred = test_pred.map(labelNormalize)

#print(train_dataset)
#test_dataset.dtypes

X = np.array(train_dataset.tolist())
Y = np.array(test_dataset.tolist())

#clf.fit(test_dataset[0].tolist(), labels)
#np.array(test_dataset).astype("float32")


In [13]:
train_pred = np.array(train_pred.tolist())
test_pred = np.array(test_pred.tolist())


In [14]:
temp=pd.DataFrame()

def augmentAfterEpoch():
    temp = train_dataset.map(augment)
    newX = np.array(temp.tolist())
    #navn.append(pd.DataFrame(temp).T)
    #navn.extend([temp])
    return newX
#print(temp)
    #a = torch.tensor(temp)
    #navn2 = torch.cat((a,navn2))

#navn = pd.merge(temp, train_pred, left_index=True, right_index=True)
#print(temp)

In [ ]:
#print(navn.shape)
#tf.convert_to_tensor(train_dataset)
print(len(train_dataset))
#type(train_pred)
plt.imshow(temp.iloc[1])
plt.imshow(navn[1])


In [ ]:
print(type(temp))

In [ ]:
print(train_pred)
#print(train_dataset)
#print(X.shape)


In [15]:
#print(test_dataset)
#df[df['price'] < 50000]
#https://www.kaggle.com/questions-and-answers/50121

#train_dataset.loc[(train_dataset["Pawpularity"]<=100)|train_dataset["Pawpularity"]>0]
#test_dataset.loc[(test_dataset["Pawpularity"]<=100)|test_dataset["Pawpularity"]>0]


#intet virker og jeg er trist del op uden merge i x-y_pred
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('petfinder.keras', save_best_only=True)
]

In [16]:
history = model.fit(X,train_pred,validation_data = (Y,test_pred),shuffle=True,batch_size=16, epochs = 30,callbacks=callbacks)
newX = augmentAfterEpoch()
history2 = model.fit(newX,train_pred,validation_data = (Y,test_pred),shuffle=True,batch_size=16, epochs = 30,callbacks=callbacks)
newX = augmentAfterEpoch()
history3 = model.fit(newX,train_pred,validation_data = (Y,test_pred),shuffle=True,batch_size=16, epochs = 30,callbacks=callbacks)

#fit(
#    x=None, y=None, batch_size=None, epochs=1, verbose='auto',
#    callbacks=None, validation_split=0.0, validation_data=None, shuffle=True,
#    class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None,
#    validation_steps=None, validation_batch_size=None, validation_freq=1,
#    max_queue_size=10, workers=1, use_multiprocessing=False
#)


Epoch 1/30
496/496 [==============================] - 12s 18ms/step - loss: 469.4054 - root_mean_squared_error: 21.6658 - val_loss: 436.1154 - val_root_mean_squared_error: 20.8834
Epoch 2/30
496/496 [==============================] - 8s 17ms/step - loss: 444.1763 - root_mean_squared_error: 21.0755 - val_loss: 423.1233 - val_root_mean_squared_error: 20.5700
Epoch 3/30
496/496 [==============================] - 8s 17ms/step - loss: 441.5261 - root_mean_squared_error: 21.0125 - val_loss: 420.8101 - val_root_mean_squared_error: 20.5137
Epoch 4/30
496/496 [==============================] - 8s 17ms/step - loss: 441.1154 - root_mean_squared_error: 21.0027 - val_loss: 434.7472 - val_root_mean_squared_error: 20.8506
Epoch 5/30
496/496 [==============================] - 8s 17ms/step - loss: 444.7563 - root_mean_squared_error: 21.0892 - val_loss: 431.6118 - val_root_mean_squared_error: 20.7753- ETA: 0s - loss: 446.0438 - ro
Epoch 6/30
496/496 [==============================] - 8s 17ms/step - loss

496/496 [==============================] - 9s 19ms/step - loss: 436.3509 - root_mean_squared_error: 20.8890 - val_loss: 428.2850 - val_root_mean_squared_error: 20.6950
Epoch 17/30
496/496 [==============================] - 9s 19ms/step - loss: 439.5786 - root_mean_squared_error: 20.9661 - val_loss: 421.2218 - val_root_mean_squared_error: 20.5237
Epoch 18/30
496/496 [==============================] - 9s 19ms/step - loss: 436.8165 - root_mean_squared_error: 20.9002 - val_loss: 423.5942 - val_root_mean_squared_error: 20.5814
Epoch 19/30
496/496 [==============================] - 9s 19ms/step - loss: 436.7179 - root_mean_squared_error: 20.8978 - val_loss: 426.5603 - val_root_mean_squared_error: 20.6533
Epoch 20/30
496/496 [==============================] - 9s 19ms/step - loss: 435.9176 - root_mean_squared_error: 20.8786 - val_loss: 421.3121 - val_root_mean_squared_error: 20.5259
Epoch 21/30
496/496 [==============================] - 9s 19ms/step - loss: 435.7007 - root_mean_squared_error: 

In [ ]:
from keras.preprocessing import image as keras_image

test_image = keras_image.load_img('train/00a1ae8867e0bb89f061679e1cf29e80.jpg', target_size = (image_size, image_size))
print(test_image)
plt.imshow(test_image)


In [17]:
print("Mean value of train "+ str(train_pred.mean()))


Mean value of train 37.98927985874637


In [18]:
display(train.head())

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,file_path
2035,348f876df5329badea38970d2497df83,0,1,1,1,0,0,0,0,0,0,0,0,77,C:\Users\tobia\GitHub\PetFinder\train\348f876d...
7552,c2cfd5bbb2826ed42f9972acb9b7f0e5,0,1,1,0,0,0,1,1,1,0,1,0,33,C:\Users\tobia\GitHub\PetFinder\train\c2cfd5bb...
6380,a4ecc2a2f3f571fc84ae7c61e1024dc7,0,1,1,1,0,0,0,0,1,1,1,0,26,C:\Users\tobia\GitHub\PetFinder\train\a4ecc2a2...
6145,9f04c5d17df0e9877e0f051da51c6843,0,1,1,1,0,0,0,0,0,0,0,0,12,C:\Users\tobia\GitHub\PetFinder\train\9f04c5d1...
3536,5b2298b9e109ef72259f48934a87d9f8,0,1,1,1,0,0,0,0,0,0,0,0,50,C:\Users\tobia\GitHub\PetFinder\train\5b2298b9...


In [19]:
#test_image = keras_image.img_to_array(test_image)
#test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(Y)
final=pd.DataFrame()
final['Id']=test['Id']
final['Subject Focus']=test['Subject Focus']
final['Eyes']=test['Eyes']
final['Face']=test['Face']
final['Near']=test['Near']
final['Action']=test['Action']
final['Accessory']=test['Accessory']
final['Group']=test['Group']
final['Collage']=test['Collage']
final['Human']=test['Human']
final['Occlusion']=test['Occlusion']
final['Info']=test['Info']
final['Blur']=test['Blur']
final['Pawpularity']=result
print(final)


                                    Id  Subject Focus  Eyes  Face  Near  \
3669  5dc85cd881e9352aff9c9114b8974d4b              0     0     0     0   
2840  490803b010323a9d62eb926d50c6d531              0     1     1     1   
149   04401eb40ba1e016aaecb5e0958ff1ab              0     1     1     0   
2395  3dac372745d2acd1371fea7f929b840a              0     0     1     1   
5862  97d9cebb29caab194d11db8917562621              0     1     1     1   
...                                ...            ...   ...   ...   ...   
506   0d4a7e3c9f0a3f45e97e885ec69185de              0     1     1     1   
3995  6620755d7add172d67214cc6f75caa8c              0     1     1     1   
2545  41e2f3d0831204725c0e9ac7ce4adde6              0     1     1     1   
8420  d9fae91ab259ce3723a7ae24d92b17ef              0     1     1     1   
7424  bf7cd72da5c2c0b2cc742ce69279f3a1              0     0     1     1   

      Action  Accessory  Group  Collage  Human  Occlusion  Info  Blur  \
3669       0          0   

In [20]:
display(final.head())

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
3669,5dc85cd881e9352aff9c9114b8974d4b,0,0,0,0,0,0,0,0,0,0,0,1,38.886623
2840,490803b010323a9d62eb926d50c6d531,0,1,1,1,0,0,0,0,0,0,0,0,38.886623
149,04401eb40ba1e016aaecb5e0958ff1ab,0,1,1,0,0,0,0,1,0,0,0,0,38.886623
2395,3dac372745d2acd1371fea7f929b840a,0,0,1,1,0,0,0,0,0,0,0,0,38.886623
5862,97d9cebb29caab194d11db8917562621,0,1,1,1,0,0,0,0,0,0,0,0,38.886623


In [ ]:
# list all data in history1
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['root_mean_squared_error'])
plt.plot(history.history['val_root_mean_squared_error'])
plt.title('model mse')
plt.ylabel('mse')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# list all data in history2
print(history.history.keys())
# summarize history for accuracy
plt.plot(history2.history['root_mean_squared_error'])
plt.plot(history2.history['val_root_mean_squared_error'])
plt.title('model mse')
plt.ylabel('mse')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# list all data in history3
print(history.history.keys())
# summarize history for accuracy
plt.plot(history3.history['root_mean_squared_error'])
plt.plot(history3.history['val_root_mean_squared_error'])
plt.title('model mse')
plt.ylabel('mse')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history3.history['loss'])
plt.plot(history3.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()